# California Housing Prices - Regressione

In questo notebook utilizzeremo una versione ridotta e leggermente modificata del dataset `California Housing Prices`. Il dataset contiene le caratteristiche delle case presenti in un determinato distretto della California e alcune statistiche riassuntive basate sul censimento statunitense del 1990. Ogni riga contiene le informazioni per uno specifico isolato residenziale (*block group*). 

Utilizzeremo questo dataset per sviluppare un modello di **regressione**, che ci permetterà di stimare il **prezzo mediano delle case** (*medianHouseValue*) in un certo isolato in base alle altre variabili socio-economiche e geografiche disponibili nel dataset. 

Dopo una prima parte di pre-processing dei dati, vedremo come implementare un modello di regressione lineare e faremo un confronto con altri modelli di regressione.

<a id="0"></a> <br>

# Indice
1. [Pre-processing dei dati](#1)
2. [Exploratory Data Analysis](#2)
3. [Implementazione e valutazione del modello di Regressione Lineare](#3)
4. [Altri modelli di regressione](#4)

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib

<a id="1"></a> <br>
## 1. Pre-processing dei dati

Lo step fondamentale prima di applicare un modello di Machine Learning è quello di studiare le caratteristiche principali dei dati per renderli utilizzabili dal modello che sceglieremo. In questa sezione, metteremo in pratica le tecniche viste nelle precedenti lezioni per analizzare il dataset, in particolare dovremo:
- caricare il dataset
- estrarre le prime descrizioni generali (dimensione, tipo di dati, variabili, ...)
- gestire i valori mancanti
- gestire variabili categoriche

- Caricamento del dataset e descrizione generale

In [ ]:
# Caricamento del dataset
df = pd.read_csv('../data/housing_modified.csv')

# Stampa le prime dieci righe
df.head(10)

In [ ]:
# Che dimensione ha il dataset? Quante righe e quante colonne ha?
df.shape

In [ ]:
# Quali sono i nomi delle variabili presenti nel dataset?
df.columns.to_list()

Il dataset contiene 10 variabili:

1. **longitude** (longitudine): Coordinata geografica che indica la longitudine della posizione dell’isolato (valori più alti indicano una posizione più occidentale)

2. **latitude** (latitudine): Coordinata geografica che indica la latitudine della posizione dell’isolato (valori più alti indicano una posizione più settentrionale)

3. **housing_median_age** (età mediana delle abitazioni): Indica l’età mediana degli edifici presenti nell'isolato; valori più bassi indicano edifici più recenti

4. **total_rooms** (numero totale di stanze): Totale delle stanze presenti in tutte le abitazioni dell’isolato

5. **total_bedrooms** (numero totale di camere da letto): Totale delle camere da letto presenti nell’isolato

6. **population** (popolazione): Numero totale di persone che risiedono nell’isolato

7. **households** (unità abitative): Numero totale di famiglie o unità abitative nell’isolato

8. **median_income** (reddito mediano): Indica il reddito mediano delle famiglie in un isolato, espresso in decine di migliaia di dollari (es. un valore di 5 equivale a 50.000$ annui)

9. **median_house_value** (valore mediano delle case): Indica il valore mediano delle abitazioni in un isolato, espresso in dollari. Questa è la variabile target che vogliamo prevedere con la regressione lineare

10. **ocean_proximity** (vicinanza all’oceano): Variabile categorica che descrive la posizione geografica dell’isolato rispetto alla costa. Può assumere i seguenti valori: "NEAR OCEAN" (vicino all’oceano), "INLAND" (nell’entroterra), "NEAR BAY" (vicino a una baia), "<1H OCEAN" (a meno di un’ora dall’oceano), "ISLAND" (situato su un’isola)

In [ ]:
# Alcune informazioni importanti sul dataset (possiamo usare il metodo .info())
df.info()

In [ ]:
# Tipo di dato in ogni colonna
df.dtypes

In [ ]:
# Caratteristiche statistiche principali per le variabili numeriche
df.describe()

- Valori mancanti

In [ ]:
# Quanti valori nulli ci sono in ogni colonna?
df.isna().sum()

In [ ]:
# Come gestiamo i valori nulli?
# Ci sono vari metodi per gestire i dati mancanti: possiamo eliminare dal dataset le righe corrispondenti, sostituirli con un valore medio o con il valore mediano, ...
# In questo caso, siccome le righe che contengono i valori mancanti per 'total_bedrooms' sono solo 3 possiamo
# decidere di eliminare direttamente le righe corrispondenti
df.dropna(inplace=True) # Eliminiamo le righe che hanno almeno un valore nullo

In [ ]:
# Controlliamo che l'operazione sia andata a buon fine
df.isna().sum()

In [ ]:
# Controlliamo se ci sono duplicati in seguito alla nostra operazione e in caso affermativo li rimuoviamo
df.duplicated().sum()

- Variabili categoriche

In [ ]:
# Analizziamo la colonna ocean_proximity contando il numero di osservazioni per ogni classe
ocean_values = df["ocean_proximity"].value_counts() # Inserire il nome della colonna di cui vogliamo analizzarei valori
ocean_values

In [ ]:
# Trasformiamo la variabile categorica 'ocean_proximity' in variabile ordinale
# Ricorda: una variabile ordinale è una variabile che assume valori categorici che però possono essere ordinati 
# (ad esempio una variabile 'Titolo di studio' con tre modalità disposte in ordine crescente: licenza media inferiore, diploma e laurea)
from sklearn.preprocessing import OrdinalEncoder

# Inizializzare l'OrdinalEncoder
encoder = OrdinalEncoder(categories=[['ISLAND','NEAR OCEAN', 'NEAR BAY','<1H OCEAN', 'INLAND']])

# Selezionare la variabile da trasformare
ocean_proximity_encoded = encoder.fit_transform(df[['ocean_proximity']])

# Convertire la colonna del DataFrame originario con i dati encoded
df['ocean_proximity'] = ocean_proximity_encoded

In [ ]:
# Stampare il DataFrame aggiornato
df.head()

<a id="2"></a> <br>

## 2. Exploratory data analysis (EDA)

In questa sezione utilizzeremo dei metodi di visualizzazione dei dati per continuare ad analizzare le caratteristiche del dataset. 
In particolare dovremo:
- plottare la correlation heatmap per valutare la correlazione tra le diverse variabili
- studiare le distribuzioni delle diverse variabili
- gestire gli outliers

- Correlazione

In [ ]:
# Stampiamo la correlation heatmap per valutare la correlazione tra le variabili
plt.figure(figsize=(10, 8))
sns.heatmap(df.corr(numeric_only=True), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Heatmap')
plt.show()

- Distribuzioni delle variabili

In [ ]:
# Plottiamo la distribuzione geografica degli isolati. In quali zone le case hanno un prezzo più alto?
df.plot(
    kind="scatter", x="longitude", y="latitude", alpha=0.4,
    s=df["population"] / 100, label="population", figsize=(15, 8),
    c="median_house_value",  colorbar=True
)
plt.show()

In [ ]:
# Studiamo la distribuzione delle variabili presenti nel dataset
df.hist(bins=25,figsize=(20,10))
plt.show()

<a id="3"></a> <br>

## 3. Implementazione e valutazione del modello di Regressione Lineare

In questa sezione costruiremo e alleneremo il modello di regressione lineare (seguendo gli step illustrati nella presentazione). Infine valuteremo il modello ottenuto calcolando diverse metriche.

LinearRegression (documentazione): https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error , mean_absolute_percentage_error , mean_squared_error

In [ ]:
# Definiamo le variabili di input (X) e di output (y)
# Il nome della colonna da prevedere è il prezzo medio delle case in un isolato ('median_house_value')
X = df.drop(columns=['...' ])
y= df['...' ]

In [ ]:
# Dividiamo i dati in training (80%) e test(20%)
...

In [ ]:
# Controlliamo la dimensione del dataset di training e di test
...
...
...
...

In [ ]:
# Standardizziamo i dati
ro_scaler=RobustScaler()
X_train=ro_scaler.fit_transform(X_train)
X_test=ro_scaler.transform(X_test)

In [ ]:
# Creiamo il modello di Regressione Lineare
model = ...

In [ ]:
# Alleniamo il modello sui dati di training
...

In [ ]:
# Facciamo previsioni sui dati di test
y_predict = ...

In [ ]:
# Valutiamo il modello

# calcoliamo le diverse metriche (R2, Mean Squared Error (MSE), Root Mean Squared Error (RMSE), Mean Absolute Error (MAE), Mean Absolupte Percentage Error (MAPE))
r_squared = r2_score(... , ...)
mse = mean_squared_error(..., ...)
rmse = np.sqrt(mse)
mae = mean_absolute_error(...,...)
mape = mean_absolute_percentage_error(... , ...) # Misura l'errore medio in percentuale rispetto ai valori reali

print('R squared (R2):',r_squared)
print(f'Mean Squared Error (MSE):{mse}')
print(f'Root Mean Squared Error (RMSE):{rmse}')
print(f'Mean Absolute Error (MAE):{mae}')
print("Mean  absolute precentage error of linear regression : ",mape*100,'%')

In [ ]:
# Plottiamo i valori reali vs i prezzi previsti. Se il modello non facesse errori dove dovrebbero essere tutti i punti?
plt.scatter(np.array(y_test), y_predict)
plt.plot(np.array(y_test),np.array(y_test),color='red')
plt.xlabel("Prezzi reali delle case")
plt.ylabel("Prezzi previsti")
plt.title("Risultati Regressione Lineare: Prezzi reali vs Previsti")
plt.show()

<a id="4"></a> <br>

## 4. Altri modelli di regressione

In questa sezione vedremo altri modelli di regressione e confronteremo gli errori con quelli ottenuti con il modello di regressione lineare. I modelli che vedremo saranno:
- Ridge regression
- Lasso regression
- XGBoost Regressor

*Ridge e Lasso Regression*
Ridge Regression e Lasso Regression sono due versioni della regressione lineare che aggiungono una penalizzazione per evitare modelli troppo complessi (overfitting).

*XGBoost Regressor*
XGBoost Regressor è un modello avanzato basato su Gradient Boosting, che crea tanti piccoli alberi decisionali e li migliora passo dopo passo, correggendo gli errori fatti dai precedenti. È veloce e gestisce bene i dati con outlier o valori mancanti.

In [ ]:
from sklearn.linear_model import Ridge, Lasso
from xgboost import XGBRegressor

In [ ]:
# Creiamo il modello di Ridge Regression
ridge = Ridge(alpha=100, random_state=42)  # Prova con diversi valori di alpha

# Alleniamo il modello sui dati di training
...

# Facciamo previsioni sui dati di test
ridge_pred = ...

In [ ]:
# Valutiamo il modello di Ridge Regression
r_squared_ridge = r2_score(y_test , ridge_pred)
mse_ridge = mean_squared_error(y_test, ridge_pred)
rmse_ridge = np.sqrt(mse_ridge)
mae_ridge = mean_absolute_error(y_test,ridge_pred)
mape_ridge = mean_absolute_percentage_error(y_test , ridge_pred)

print('R squared of linear regression :',r_squared_ridge)
print(f'Mean Squared Error (MSE):{mse_ridge}')
print(f'Root Mean Squared Error (RMSE):{rmse_ridge}')
print(f'Mean Absolute Error (MAE):{mae_ridge}')
print("Mean  absolute precentage error of linear regression : ",mape_ridge*100,'%')

In [ ]:
# Creiamo il modello di Lasso Regression
lasso = Lasso(alpha=100, random_state=42)

# Alleniamo il modello sui dati di training
...

# Facciamo previsioni sui dati di test
lasso_pred = ...

In [ ]:
r_squared_lasso = r2_score(y_test , lasso_pred)
mse_lasso = mean_squared_error(y_test, lasso_pred)
rmse_lasso = np.sqrt(mse_lasso)
mae_lasso = mean_absolute_error(y_test,lasso_pred)
mape_lasso = mean_absolute_percentage_error(y_test , lasso_pred)

print('R squared of linear regression :',r_squared_lasso)
print(f'Mean Squared Error (MSE):{mse}')
print(f'Root Mean Squared Error (RMSE):{rmse}')
print(f'Mean Absolute Error (MAE):{mae}')
print("Mean  absolute precentage error of linear regression : ",mape_lasso*100,'%')

In [ ]:
# Creiamo il modello di XGBoost Regressor
xgb = XGBRegressor(n_estimators=80, learning_rate=0.1, random_state=42)

# Alleniamo il modello sui dati di training
...

# Facciamo previsioni sui dati di test
xgb_pred = ...

In [ ]:
r_squared_xgb = r2_score(y_test , xgb_pred)
mse_xgb = mean_squared_error(y_test, xgb_pred)
rmse_xgb = np.sqrt(mse_xgb)
mae_xgb = mean_absolute_error(y_test,xgb_pred)
mape_xgb = mean_absolute_percentage_error(y_test , xgb_pred)

print('R squared (R2) :',r_squared_lasso)
print(f'Mean Squared Error (MSE):{mse_xgb}')
print(f'Root Mean Squared Error (RMSE):{rmse_xgb}')
print(f'Mean Absolute Error (MAE):{mae_xgb}')
print("Mean  absolute precentage error of linear regression : ",mape_xgb*100,'%')

Qual è il modello migliore?

In [ ]:
# Confrontiamo gli errori ottenuti con i diversi modelli

# Creiamo un dizionario con gli errori ottenuti dai diversi modelli
data = {
    "Metrica": [
        "Linear Regression",
        "Ridge Regression",
        "Lasso Regression",
        "XGBoost Regressor"
    ],
    "R²": [r_squared, r_squared_ridge, r_squared_lasso, r_squared_xgb], # più il valore è vicino a 1, migliore è il modello
    "RMSE": [rmse, rmse_ridge, rmse_lasso, rmse_xgb],
    "MAE": [mae, mae_ridge, mae_lasso, mae_xgb],
    "MAPE": [f"{mape*100}%", f"{mape_ridge*100}%", f"{mape_lasso*100}%", f"{mape_xgb*100}%"]
}

# Creiamo il DataFrame
df = pd.DataFrame(data).round(2)

# Stampiamo la tabella con un formato leggibile
print(df.to_string(index=False))

In [ ]:
plt.scatter(np.array(y_test), xgb_pred)
plt.plot(np.array(y_test),np.array(y_test),color='red')
plt.xlabel("Prezzi reali delle case")
plt.ylabel("Prezzi previsti")
plt.title("Risultati Regressione Lineare: Prezzi reali vs Previsti")
plt.show()